In [1]:
import pandas as pd
import numpy as np
import math
import requests
import geopandas as gpd

liste_med_ban = pd.read_csv('liste_med_ban.csv')

C:\Users\agath\AppData\Local\Temp\ipykernel_28948\1399691685.py:7: DtypeWarning: Columns (28,29,30,31,32,35,41,42,43,44,45,46,50) have mixed types. Specify dtype option on import or set low_memory=False.
  liste_med_ban = pd.read_csv('liste_med_ban.csv')


# Accéder à la distance par requête API

In [269]:
def cp_et_nom(cp,nom):
    if isinstance(cp,int) and isinstance(nom,str):
        return (str(cp)+' '+nom)
    else:
        return np.nan

In [270]:
liste_med_ban['cp_ville'] = liste_med_ban.apply(lambda x: cp_et_nom(x['codepostalcoordstructure'],x['libellécommunecoordstructure']),axis=1)

In [271]:
def transfo_adresse(CodePostal,NumeroVoie,NomVoie):
    num = (int(NumeroVoie))
    return (str(num)+', '+str(NomVoie)+', '+str(CodePostal))

liste_med_ban['adresse'] = liste_med_ban.apply(lambda x: transfo_adresse(x['cp_ville'],x['numérovoiecoordstructure'],x['Voie']),axis=1)

In [272]:
API_URL = "https://api-adresse.data.gouv.fr/{}/"

def req_to(endpoint: str, params: dict | None = None, files: dict | None = None) -> dict:
    url = API_URL.format(endpoint)

    if not files:
        response = requests.get(url, params=params)

    else:
        response = requests.post(url, files=files, data=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Request failed with code : {response.status_code}")


def get_geocode(**kwargs) -> dict:
    if not 'q' in kwargs:
        raise ValueError('Il manque un argument obligatoire : q')
    
    return req_to("search", kwargs)



In [273]:
numero=str(int(liste_med_ban['numérovoiecoordstructure'].iloc[0]))
voie = liste_med_ban['Voie'].iloc[0]
cp_ville=liste_med_ban['cp_ville'].iloc[0]
adresse = numero+', '+voie+', '+cp_ville
print(adresse)

# Requête simple directe:
res = get_geocode(q = adresse)
print(res)

52, rue jean jaures, 97110 Pointe-à-Pitre
{'type': 'FeatureCollection', 'version': 'draft', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-61.536772, 16.23989]}, 'properties': {'label': '52 Rue Jean Jaures 97110 Pointe-à-Pitre', 'score': 0.9688018181818181, 'housenumber': '52', 'id': '97120_0496_00052', 'banId': '3565ae2f-5cbf-4aab-8543-7a3fda9b9059', 'name': '52 Rue Jean Jaures', 'postcode': '97110', 'citycode': '97120', 'x': 656379.35, 'y': 1796029.48, 'city': 'Pointe-à-Pitre', 'context': '971, Guadeloupe', 'type': 'housenumber', 'importance': 0.65682, 'street': 'Rue Jean Jaures'}}], 'attribution': 'BAN', 'licence': 'ETALAB-2.0', 'query': '52, rue jean jaures, 97110 Pointe-à-Pitre', 'limit': 5}


In [274]:
def distance_long_lat(lat1, lon1, lat2, lon2):
    # Rayon de la Terre en km
    R = 6371.0
    
    # Conversion des degrés en radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calcul des différences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Formule de Haversine
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distance
    distance = R * c
    
    return distance

# Exemple d'utilisation
lat1, lon1 = 48.8566, 2.3522  # Paris
lat2, lon2 = 34.0522, -118.2437  # Los Angeles

distance = distance_long_lat(lat1, lon1, lat2, lon2)
print(f"La distance entre Paris et Los Angeles est d'environ {distance:.2f} km.")

La distance entre Paris et Los Angeles est d'environ 9085.51 km.


In [275]:
def recup_long(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][0])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)
    
def recup_lat(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][1])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)
#liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)

#liste_med_ban['latitude API'] = get_geocode(q = liste_med_ban['adresse'])['features'][0]['geometry']['coordinates'][1]

#liste_med_ban['adresse','longitude API','latitude API']

# Utilisation base de données centroides

In [2]:
gdf = gpd.read_file('data/a-com2022-topo-2154.json')
gdf.head()

c:\miniconda\Lib\site-packages\pyogrio\geopandas.py:261: UserWarning: More than one layer found in 'a-com2022-topo-2154.json': 'a_com2022' (default), 'a_reg2022', 'a_dep2022'. Specify layer parameter to avoid this warning.
  result = read_func(


,id,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,None,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((846981.436 6564107.36, 847017.457 65..."
1,None,01002,01,84,887495,6548152,L'Abergement-de-Varey,"POLYGON ((886027.651 6548170.9, 886157.325 654..."
2,None,01004,01,84,882724,6542583,Ambérieu-en-Bugey,"POLYGON ((884889.404 6539629.931, 884795.75 65..."
3,None,01005,01,84,847277,6545791,Ambérieux-en-Dombes,"POLYGON ((845951.25 6547999.078, 846073.72 654..."
4,None,01006,01,84,902191,6519791,Ambléon,"POLYGON ((901336.361 6521123.305, 902460.201 6..."


In [3]:
def codecom2(codecom1):
    if str(codecom1).startswith('75'):
        return('75056')
    elif str(codecom1).startswith('693'):
        return('69123')
    elif str(codecom1).startswith('132'):
        return('13055')
    elif str(codecom1).startswith('91182'):
        return('91228')
    elif str(codecom1).startswith('85257'):
        return('85090')
    elif str(codecom1).startswith('85217'):
        return('85146')
    elif str(codecom1).startswith('85166'):
        return('85194')
    elif str(codecom1).startswith('74011'):
        return('74010')
    elif str(codecom1).startswith('74181'):
        return('74112')
    elif str(codecom1).startswith('74268') or str(codecom1).startswith('74093') or str(codecom1).startswith('74217'):
        return('74010')
    elif str(codecom1).startswith('50203'):
        return('50129')
    elif str(codecom1).startswith('59540'):
        return('59139')
    elif str(codecom1).startswith('49199'):
        return('71291')
    elif str(codecom1).startswith('29052'):
        return('29003')
    elif str(codecom1).startswith('85060'):
        return('85194')
    elif str(codecom1).startswith('50602'):
        return('50129')
    elif str(codecom1).startswith('49285'):
        return('49301')
    elif str(codecom1).startswith('85107'):
        return('85146')
    elif str(codecom1).startswith('50173'):
        return('50129')
    elif str(codecom1).startswith('49065'):
        return('49080')   
    elif str(codecom1).startswith('49375'):
        return('49023')
    elif str(codecom1).startswith('85069'):
        return('85008')  
    elif str(codecom1).startswith('85069'):
        return('85008')   
    elif str(codecom1).startswith('49276'):
        return('49244') 
    elif str(codecom1).startswith('49273'):
        return('49301')    
    elif codecom1//10000==0:
        return('0'+str(codecom1))
    else :
        return(str(codecom1))

In [4]:
liste_med_ban['codecommunecoordstructure2']=liste_med_ban['codecommunecoordstructure'].apply(codecom2)

In [5]:
liste_med_ban['Code Département'] = liste_med_ban['Code Département'].astype(str)

In [6]:
liste_med_ban_metr = liste_med_ban[~liste_med_ban['Code Département'].str.startswith('971')]

In [7]:
def nombre_de_med(codeCom):
    data_f = liste_med_ban_metr[(liste_med_ban_metr['codecommunecoordstructure']==codeCom)]
    return(data_f.shape[0])

#liste_med_ban_metr['nb medecin commune']=liste_med_ban_metr['codecommunecoordstructure'].apply(nombre_de_med)

In [8]:
df_merged_gdf = liste_med_ban_metr.merge(gdf, how='inner', left_on='codecommunecoordstructure2', right_on='codgeo')

In [9]:
merged_df = liste_med_ban_metr.merge(gdf, how='outer', left_on='codecommunecoordstructure2', right_on='codgeo', indicator = True)

# Filtrer les lignes qui n'ont pas de correspondance
missing_from_df2 = merged_df[merged_df['_merge'] == 'left_only']
missing_from_df1 = merged_df[merged_df['_merge'] == 'right_only']

In [10]:
missing_from_df2['codecommunecoordstructure2']

16434    22038
16435    22038
16436    22038
16437    22038
16438    22038
81366    77491
81367    77491
81368    77491
81369    77491
81370    77491
Name: codecommunecoordstructure2, dtype: object

In [11]:
print(df_merged_gdf.shape,liste_med_ban_metr.shape)

(71906, 86) (71916, 78)


In [13]:
df_merged_gdf = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle',
       'codesavoirfaire', 'libellésavoirfaire', 'codemodeexercice',
       'numérosiretsite',
       'identifianttechniquedelastructur', 'raisonsocialesite',
       'numérovoiecoordstructure', 'indicerépétitionvoiecoordstructu',
       'codetypedevoiecoordstructure', 'libellétypedevoiecoordstructure',
       'libellévoiecoordstructure', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'libellépayscoordstructure',
       'codesecteurdactivité', 'libellésecteurdactivité', 'coderôle',
       'libellérôle', 'Doctor Name', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Travaux et publications b', 'Nb langues',
       'Expériences b', 'Formations b', 'Carac_card_1', 'Transport', 'Parking',
       'Infos', 'Coordonnées', 'Sans RDV', 'nb terms card_0',
       'nb terms card_3', 'nb terms card_4', 'nb carac card_0',
       'nb carac card_3', 'nb carac card_4', 'code dépar', 'Code Département',
       'généralistes', 'Voie', 'codgeo', 'xcl2154',
       'ycl2154','Population totale',"Population standardisée par la consommation de soins par tranche d'âge"]]

In [15]:
df_merged_gdf_reduit = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle', 'libellésavoirfaire', 'codemodeexercice', 'libellésecteurdactivité','codecommunecoordstructure',
       'libellérôle', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Travaux et publications b', 'Nb langues',
       'Expériences b', 'Formations b', 'Carac_card_1', 'Transport', 'Parking',
       'Infos', 'Coordonnées', 'Sans RDV', 'nb terms card_0',
       'nb terms card_3', 'nb terms card_4', 'nb carac card_0',
       'nb carac card_3', 'nb carac card_4', 'Code Département',
       'généralistes', 'codgeo', 'xcl2154',
       'ycl2154','Population totale',"Population standardisée par la consommation de soins par tranche d'âge"]]

In [16]:
df_merged_gdf_reduit_2 = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'RPPS', 'Code Département', 'codgeo', 'xcl2154',
       'ycl2154','codecommunecoordstructure','Population totale',"Population standardisée par la consommation de soins par tranche d'âge"]]

In [17]:
def distance_bis(x1,y1,x2,y2):
    return((np.sqrt((x1-x2)**2+(y1-y2)**2))/1000)

In [19]:
def nbDocDist(listeDist,data):
    longueur = data.shape[0]
    nbListeDoctolib = [0 for i in range(len(listeDist))]
    nbListeTot = [0 for i in range(len(listeDist))]
    nbListeFemmeTot = [0 for i in range(len(listeDist))]
    nbListeFemmeDoc = [0 for i in range(len(listeDist))]
    for (j,distanceMax) in enumerate(listeDist) : 
        for i in range(longueur):
            if data['distance'].iloc[i]<distanceMax :
                nbListeTot[j]=nbListeTot[j]+1
                if data['RPPS_y'].iloc[i]>0 :
                    nbListeDoctolib[j]=nbListeDoctolib[j]+1
                    if data['codecivilité_y'].iloc[i]!='M':
                        nbListeFemmeDoc[j]=nbListeFemmeDoc[j]+1
                if data['codecivilité_y'].iloc[i]!='M':
                    nbListeFemmeTot[j]=nbListeFemmeTot[j]+1

        data['Nb docteurs à '+str(distanceMax)+' km']=nbListeTot[j]
        data['Nb docteurs femme à '+str(distanceMax)+' km']=nbListeFemmeTot[j]
        data['Nb docteurs sur Doctolib à '+str(distanceMax)+' km']=nbListeDoctolib[j]
        data['Nb docteurs femme sur Doctolib à '+str(distanceMax)+' km']=nbListeFemmeDoc[j]

    return data[['identifiantpp_x','codecivilité_x','nomdexercice_x','prénomdexercice_x','codecatégorieprofessionnelle_x','libellésavoirfaire_x','codemodeexercice_x','libellésecteurdactivité_x','libellérôle_x','Specialty_x', 'card_0_x', 'card_3_x', 'card_4_x',
       'Latitude_x', 'Longitude_x', 'Skills Processed_x', 'RPPS_x','Nb Skills_x','moyens_paiement_x', 'carte_vitale_x', 'secteur_x', 'Présentation_x',
       'Langues parlées_x', 'Diplômes nationaux et universitaires_x',
       'Autres formations_x', 'Expériences_x', 'Travaux et publications_x',
       'Prix et distinctions_x','site_x', 'Site web_x', 'Formations_x',
       'Diplômes nationaux et universitaires b_x',
       'Nb caractères présentation_x', 'Autres formations b_x',
       'Travaux et publications b_x', 'Nb langues_x', 'Expériences b_x',
       'Formations b_x','Carac_card_1_x', 'Transport_x', 'Parking_x', 'Infos_x',
       'Coordonnées_x', 'Sans RDV_x', 'nb terms card_0_x', 'nb terms card_3_x',
       'nb terms card_4_x', 'nb carac card_0_x', 'nb carac card_3_x',
       'nb carac card_4_x', 'codgeo_x', 'xcl2154_x', 'ycl2154_x','Nb docteurs à 2 km','Nb docteurs à 8 km','Nb docteurs à 30 km','Nb docteurs femme à 2 km','Nb docteurs femme à 8 km','Nb docteurs femme à 30 km','Nb docteurs sur Doctolib à 2 km','Nb docteurs sur Doctolib à 8 km','Nb docteurs sur Doctolib à 30 km','Nb docteurs femme sur Doctolib à 2 km','Nb docteurs à 8 km','Nb docteurs à 30 km']].iloc[0]    


def nbDocDist2(listeDist,data):
    longueur = data.shape[0]
    nbListeDoctolib = [0 for i in range(len(listeDist))]
    nbListeTot = [0 for i in range(len(listeDist))]
    nbListeFemmeTot = [0 for i in range(len(listeDist))]
    nbListeFemmeDoc = [0 for i in range(len(listeDist))]
    for (j,distanceMax) in enumerate(listeDist) : 
        for i in range(longueur):
            if data['distance'].iloc[i]<distanceMax :
                nbListeTot[j]=nbListeTot[j]+1
                if data['RPPS_y'].iloc[i]>0 :
                    nbListeDoctolib[j]=nbListeDoctolib[j]+1
                    if data['codecivilité_y'].iloc[i]!='M':
                        nbListeFemmeDoc[j]=nbListeFemmeDoc[j]+1
                if data['codecivilité_y'].iloc[i]!='M':
                    nbListeFemmeTot[j]=nbListeFemmeTot[j]+1

        data['Nb docteurs à '+str(distanceMax)+' km']=nbListeTot[j]/data['Population totale_x'].iloc[0]
        data['Nb docteurs femme à '+str(distanceMax)+' km']=nbListeFemmeTot[j]/data['Population totale_x'].iloc[0]
        data['Nb docteurs sur Doctolib à '+str(distanceMax)+' km']=nbListeDoctolib[j]/data['Population totale_x'].iloc[0]
        data['Nb docteurs femme sur Doctolib à '+str(distanceMax)+' km']=nbListeFemmeDoc[j]/data['Population totale_x'].iloc[0]

    return data[['identifiantpp_x','codecivilité_x','nomdexercice_x','prénomdexercice_x', 'RPPS_x',
        'codgeo_x', 'xcl2154_x', 'ycl2154_x','Nb docteurs à 2 km','Nb docteurs à 8 km','Nb docteurs à 30 km','Nb docteurs femme à 2 km','Nb docteurs femme à 8 km','Nb docteurs femme à 30 km','Nb docteurs sur Doctolib à 2 km','Nb docteurs sur Doctolib à 8 km','Nb docteurs sur Doctolib à 30 km','Nb docteurs femme sur Doctolib à 2 km','Nb docteurs à 8 km','Nb docteurs à 30 km']].iloc[0]           

In [288]:
# Jointure croisée avec 'how=cross'
df_cross_0 = pd.merge(df_merged_gdf_reduit[df_merged_gdf_reduit['identifiantpp']==df_merged_gdf_reduit['identifiantpp'].iloc[1]], df_merged_gdf_reduit, how='cross')
df_cross_0.head()

,identifiantpp_x,codecivilité_x,nomdexercice_x,prénomdexercice_x,codecatégorieprofessionnelle_x,libellésavoirfaire_x,codemodeexercice_x,libellésecteurdactivité_x,codecommunecoordstructure_x,libellérôle_x,...,nb terms card_3_y,nb terms card_4_y,nb carac card_0_y,nb carac card_3_y,nb carac card_4_y,Code Département_y,généralistes_y,codgeo_y,xcl2154_y,ycl2154_y
0,10000011808,MME,HAMOT,ENNA,C,Qualifié en Médecine Générale,S,Centres de santé,75110,Médecin du travail,...,NaN,NaN,NaN,NaN,NaN,95,95,95637,628442,6881566
1,10000011808,MME,HAMOT,ENNA,C,Qualifié en Médecine Générale,S,Centres de santé,75110,Médecin du travail,...,NaN,NaN,NaN,NaN,NaN,75,234,75056,652492,6862009
2,10000011808,MME,HAMOT,ENNA,C,Qualifié en Médecine Générale,S,Centres de santé,75110,Médecin du travail,...,NaN,NaN,NaN,NaN,NaN,11,143,11304,633310,6197645
3,10000011808,MME,HAMOT,ENNA,C,Qualifié en Médecine Générale,S,Centres de santé,75110,Médecin du travail,...,NaN,NaN,NaN,NaN,NaN,56,178,56034,243618,6738059
4,10000011808,MME,HAMOT,ENNA,C,Qualifié en Médecine Générale,S,Centres de santé,75110,Médecin du travail,...,NaN,NaN,NaN,NaN,NaN,93,99,93048,658989,6862633


dico_cross = []
dico_d={}
for i in range(5000):
    df_cross = pd.merge(df_merged_gdf_reduit[(df_merged_gdf_reduit['identifiantpp']==df_merged_gdf_reduit['identifiantpp'].iloc[i])&(df_merged_gdf_reduit['codecommunecoordstructure']==df_merged_gdf_reduit['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit[df_merged_gdf_reduit['Code Département']==df_merged_gdf_reduit['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist([2,8,30],df_cross)
    dico_cross.append(ligne)
    #print(i)

In [261]:
dico_cross[199]

identifiantpp_x                                      10000189661
codecivilité_x                                                 M
nomdexercice_x                                         CHAUCHARD
prénomdexercice_x                                         CLAUDE
codecatégorieprofessionnelle_x                                 C
libellésavoirfaire_x                           Médecine Générale
codemodeexercice_x                                             L
libellésecteurdactivité_x                     Cabinet individuel
libellérôle_x                               Titulaire de cabinet
Specialty_x                                                  NaN
card_0_x                                                     NaN
card_3_x                                                     NaN
card_4_x                                                     NaN
Latitude_x                                                   NaN
Longitude_x                                                  NaN
Skills Processed_x       

In [20]:
dico_cross2 = []
dico_d={}
for i in range(5000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)
    #print(i)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\miniconda\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\agath\AppData\Local\Temp\ipykernel_28948\155578252.py", line 18, in <module>
    distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
                                                   ^^^^^^^^^^
NameError: name 'df_cross_0' is not defined. Did you mean: 'df_cross'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\miniconda\Lib\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\miniconda\Lib\site-packages\IPython\core\ultratb.py", line 1454, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^

In [296]:
for i in range(5000,10000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/3971064186.py", line 9, in <module>
    codegeo2 = df_cross['codgeo_y'].iloc[i]
               ~~~~~~~~^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/frame.py", line 4078, in __getitem__
    return self._get_item_cache(key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/frame.py", line 4639, in _get_item_cache
    res = self._ixs(loc, axis=1)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/frame.py", line 4014, in _ixs
    result._set_as_cached(label, self)
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/series.py", line 1471, i

In [ ]:
for i in range(10000,20000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(20000,30000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(35000,40000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(40000,45000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(45000,50000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(50000,55000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(55000,60000):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

In [ ]:
for i in range(60000,df_merged_gdf_reduit_2.shape[0]):
    df_cross = pd.merge(df_merged_gdf_reduit_2[(df_merged_gdf_reduit_2['identifiantpp']==df_merged_gdf_reduit_2['identifiantpp'].iloc[i])&(df_merged_gdf_reduit_2['codecommunecoordstructure']==df_merged_gdf_reduit_2['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit_2[df_merged_gdf_reduit_2['Code Département']==df_merged_gdf_reduit_2['Code Département'].iloc[i]], how='cross')
    df_cross['distance']=0.
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist2([2,8,30],df_cross)
    dico_cross2.append(ligne)

# Merge avec les données de Doctolib

In [ ]:
medecin_gen = pd.read_csv('medecin_gen.csv')

Transformer la liste en une dataframe pour pouvoir la joindre à la base doctolib

In [ ]:
df_form_cross = pd.DataFrame(dico_cross)
print(df_form_cross)

### API OpenRouteService

In [ ]:
import openrouteservice

# Clé API OpenRouteService
client = openrouteservice.Client(key='5b3ce3597851110001cf6248fa09b3e419ef40e182777370274818e9')

# Coordonnées (longitude, latitude) pour le départ et l'arrivée
coordinates = [[2.3522, 48.8566], [4.8357, 45.764]]

# Calcul de l'itinéraire
route = client.directions(coordinates, profile='driving-car')

# Temps de trajet en secondes
duration = route['routes'][0]['summary']['duration']
print(f"Temps de trajet : {duration / 60} minutes")